In [1]:
import os
import re
import json
import math
import pandas as pd

In [2]:
data_root = 'raw_data'
data_files = [f'{journal}_journals.csv'for journal in ['ce', 'di', 'wu', 'yao', 'yaoxi']]


In [3]:
def min_edit_distance(text_a, text_b):
    '''计算两字符串(列表)之间的最小编辑距离
    source: https://leetcode-cn.com/problems/edit-distance/solution/bian-ji-ju-chi-by-leetcode-solution/
    '''
    n = len(text_a)
    m = len(text_b)
    # 有一个字符串为空串
    if n * m == 0:
        return n + m
    # DP 数组
    D = [ [0] * (m + 1) for _ in range(n + 1)]
    # 边界状态初始化
    for i in range(n + 1):
        D[i][0] = i
    for j in range(m + 1):
        D[0][j] = j
    # 计算所有 DP 值
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            left = D[i - 1][j] + 1
            down = D[i][j - 1] + 1
            left_down = D[i - 1][j - 1] 
            if text_a[i - 1] != text_b[j - 1]:
                left_down += 1
            D[i][j] = min(left, down, left_down)
    
    return D[n][m]

In [48]:
split_pattern = re.compile(r'[;,/\s+]') # 关键词间可能的 分隔符
colon_pattern = re.compile(r'[:：]') # 中英文模式下的 冒号
space_pattern = re.compile(r'\s+')
zh_pattern = re.compile(u'[\u4e00-\u9fa5]+')
def split_keywords(keywords: str):
    '''str to list，去掉“关键词”、各种分隔符和空格'''
    # 去掉“关键词”字眼
    split_colon = colon_pattern.split(keywords)[-1]
    split_res = split_pattern.split(split_colon)
    keyword_list = []
    for keyword in split_res:
        keyword = keyword.strip()
        if len(keyword) > 0:
            keyword_list.append(keyword)
            
    return keyword_list
def match_keyword_prefix(keyword, text, len_diff_ratio_thres=0.5):
    matched_kw = ''
    kw_len = len(keyword)
    title_len = len(text)
    min_edit_dis = math.ceil(len_diff_ratio_thres * len(keyword))
    for start_ind in range(title_len-kw_len+2):
        if keyword[0] != text[start_ind]: continue # 先保证第一字匹配上
        for candidate_len in range(min(2, kw_len-min_edit_dis), kw_len+min_edit_dis): # 每个关键词的长度都大于 2
            end_ind = start_ind + candidate_len - 1
            end_ind = min(len(text)-1, end_ind)
            # 至少第一字和最后一个字是匹配上的
            candidate_kw = text[start_ind: end_ind+1]
            # if keyword[-1] == text[end_ind]:
            #     matched_kw = candidate_kw
            #     break
            # 增加根据编辑距离寻找匹配关键词的步骤
            edit_dis = min_edit_distance(keyword, candidate_kw)
            if edit_dis < min_edit_dis:
                #if len(candidate_kw) >= len(matched_kw):
                matched_kw = candidate_kw
                # print(matched_kw)
                # if matched_kw[-1] == keyword[-1]:
                #     break
                min_edit_dis = edit_dis

                # print(candidate_kw, min_edit_dis)
        if matched_kw is not None and len(matched_kw) > 0:
            break
    return matched_kw

def match_keyword_suffix(keyword, text, len_diff_ratio_thres=0.5):
    matched_kw = ''
    kw_len = len(keyword)
    title_len = len(text)
    min_edit_dis = math.ceil(len_diff_ratio_thres * len(keyword))
    for start_ind in range(title_len-1, 0, -1):
        if keyword[-1] != text[start_ind]: continue # 先保证最后一字匹配上
        
        for candidate_len in range(min(2, kw_len-min_edit_dis), kw_len+min_edit_dis): # 每个关键词的长度都大于 2
            end_ind = start_ind - candidate_len
            if end_ind < -1 or end_ind == start_ind: continue
            
            candidate_kw = text[end_ind+1: start_ind+1]
            # print(kw_len, start_ind, end_ind)
            # if keyword[0] == text[end_ind+1]:
            #     matched_kw = candidate_kw
            #     break
            # 增加根据编辑距离寻找匹配关键词的步骤
            edit_dis = min_edit_distance(keyword, candidate_kw)
            if edit_dis < min_edit_dis:
                #if len(candidate_kw) >= len(matched_kw):
                matched_kw = candidate_kw
                # if matched_kw[0] == keyword[0]:
                #     break
                min_edit_dis = edit_dis
                # print(candidate_kw, min_edit_dis)
        if matched_kw is not None and len(matched_kw) > 0:
            break
    return matched_kw

def match_keywords(keyword_list, text, len_diff_ratio_thres=0.5):
    '''从 keyword_list 中找到能在 text 中出现的 keyword，组成新的 keyword_list'''
    matched_set = set()
    text = text.strip()
    for keyword in keyword_list:
        keyword = keyword.strip()
        keyword_len = len(keyword)
        if keyword_len < 2: continue # 只考虑字数大于等于 2 的关键词
        matched_kw = match_keyword_prefix(keyword, text, len_diff_ratio_thres)
        if len(matched_kw) > 0:
            matched_set.add(matched_kw)
            if matched_kw != keyword:
                print(f'前缀找到了：{keyword}->{matched_kw}, {text}')
            #matched_list.append(matched_kw)
        else:
            matched_kw = match_keyword_suffix(keyword, text, len_diff_ratio_thres)
            if len(matched_kw) > 0:
                matched_set.add(matched_kw)
                if matched_kw != keyword:
                    print(f'后缀找到了：{keyword}->{matched_kw}, {text}')
    return list(matched_set)

def preprocess(data_path, save_json_path=None, file_mode='w'):
    
    data_frame = pd.read_csv(data_path, usecols=['title', 'keywords'])
    with open(save_json_path, file_mode, encoding='utf-8') as writer:
        for index, row in data_frame.iterrows():
            title, keywords = row['title'], row['keywords']
            title = space_pattern.sub('', title)
            # 只针对中文标题
            if not zh_pattern.search(title):
                print(title)
                continue
            keyword_list = split_keywords(keywords)
            keyword_matched = match_keywords(keyword_list, title)
            one_dict = {'title': title, 'keywords_matched': keyword_matched, 'keywords': keyword_list}
            writer.write(json.dumps(one_dict, ensure_ascii=False))
            writer.write('\n')

In [5]:
keyword_list = ['LiDAR', '点云分类', 'JointBoost', '空间上下文', '特征降维度']
title = '顾及空间上下文关系的JointBoost点云分类及特征降维'
matched_list = match_keywords(keyword_list, title, precise_mode=False, len_diff_ratio_thres=0.5)
print(matched_list)

找到了：特征降维度->类及特征降维, 顾及空间上下文关系的JointBoost点云分类及特征降维
['点云分类', 'JointBoost', '空间上下文', '类及特征降维']


In [49]:

for i, data_f in enumerate(data_files):
    data_path = os.path.join(data_root, data_f)
    #data_frame = pd.read_csv(data_path, usecols=['title', 'keywords'])
    file_mode = 'w' if i == 0 else 'a'
    preprocess(data_path, 'journals/preprocessed_data.jl', file_mode= file_mode)
    

前缀找到了：冰川泥石流->冰川流, 冰川流域孕灾环境及灾害的天空地协同智能监测模式与方向
后缀找到了：新一代人工智能->人工智能, 测绘地理信息与人工智能2.0融合发展的方向
前缀找到了：时空大数据地图表达->时空数据地图表达, 时空数据地图表达的基本问题与研究进展
前缀找到了：地图表达自适化->地图表达, 时空数据地图表达的基本问题与研究进展
前缀找到了：遥感智能解译->遥感影像智能解译, 遥感影像智能解译样本库现状与研究
后缀找到了：多源遥感影像->遥感影像, 遥感影像智能解译样本库现状与研究
后缀找到了：时空多尺度->多尺度, 多尺度GTWR城市住宅价格建模与分析
后缀找到了：标线矢量化->矢量化, 车载激光点云中交通标线自动分类与矢量化
前缀找到了：星载全波形激光->星载激光, 全波形星载激光测距误差抑制的滑动窗口高斯拟合算法
前缀找到了：三维地表形变->三维同震地表形变, InSAR三维同震地表形变监测——窗口优化的SM-VCE算法
前缀找到了：交通信号控制->交通信号, 面向地理路网的交通信号智能协同控制方法
前缀找到了：地理知识服务->地理信息服务, 从地理信息服务到地理知识服务:基本问题与发展路径
前缀找到了：时空知识库->时空知识, 时空知识中心的研究进展与应用
前缀找到了：地理空间科学->地理空间, 地理空间智能研究进展和面临的若干挑战
前缀找到了：空间异常区域->空间异常, 流空间邻近关系约束下的流行病分布空间异常探测方法
前缀找到了：朴素贝叶斯分类器->朴素贝叶斯, 地图线状要素眼动识别的朴素贝叶斯方法
前缀找到了：地图学史->地图学, 空间认知推动地图学学科发展的新方向
前缀找到了：结构张量->结合张量, 结合张量与互信息的混合模型多模态图像配准方法
后缀找到了：积雪深度->雪深度, 干雪深度反演的同极化相位差模型
前缀找到了：业务化测绘->业务化, SAR卫星业务化地形测绘能力分析与展望
后缀找到了：重采样->采样, 非线性模型精度评定的Bootstrap方法及其加权采样改进方法
前缀找到了：边界入射信号->边界信号, 顾及边界信号及垂直约束的GNSS水汽层析方法
前缀找到了：珠峰高程测量->珠峰测量, 2020珠峰测量与高程确定
前缀找到了：多波束测深->多波束, 顾及传播曲面的多波束波束脚印高精度快速归位算法
前缀找到了：传播曲面模型